In [1]:
# !pip install webdriver_manager
# !apt-get update
# !apt install chromium-chromedriver --yes
# !pip install selenium
# !pip install ipywidgets

In [ ]:
from tqdm import notebook
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [70]:
def chrome_setting():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)
  return driver

In [71]:
driver=chrome_setting()

In [72]:
import random

In [73]:
# Crawling
def back_trans(text_data, lang):
    try:
        target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
        driver.get('https://papago.naver.com/?sk=ko&tk='+lang+'&st='+text_data)
        time.sleep(random.uniform(0.2, 1))
        element=WebDriverWait(driver, 10).until(target_present)
        time.sleep(random.uniform(0.2, 1))
        backtrans = element.text
    except:
        print(text_data)
        return ""
    
    try:
        target_present = EC.presence_of_element_located((By.XPATH, '//*[@id="txtTarget"]'))
        driver.get('https://papago.naver.com/?sk='+lang+'&tk=ko&st='+backtrans)
        time.sleep(random.uniform(0.2, 1))
        element=WebDriverWait(driver, 10).until(target_present)
        time.sleep(random.uniform(0.2, 1))
        new_backtrans = element.text
        return new_backtrans
    except:
        print(text_data)
        return ""

In [ ]:
# 테스트 영어/중국어/일본어 back-translation
print(back_trans("열린우리당을 흡수합당한 대통합민주신당과 민주당과의 합당때 합류하지 않은 친노직계 세력이 주축이 되어 2010년 1월 17일 창당한 정당이다.","en"))
print(back_trans("열린우리당을 흡수합당한 대통합민주신당과 민주당과의 합당때 합류하지 않은 친노직계 세력이 주축이 되어 2010년 1월 17일 창당한 정당이다.","zh-CN"))
print(back_trans("열린우리당을 흡수합당한 대통합민주신당과 민주당과의 합당때 합류하지 않은 친노직계 세력이 주축이 되어 2010년 1월 17일 창당한 정당이다.","ja"))

## 외국어인지 판별하는 함수

In [59]:
def find_foreign(start: int, end: int, sentence: str):
    count=0
    a = int(start, 16) 
    b = int(end, 16)
    return_sentence = ''
    for i, w in enumerate(sentence):
        if a <= ord(w) and ord(w) <= b:
            return True
    return False

## 품질 좋은 역번역 데이터 만들기
### 1. 중복 제거, 괄호 제거 전체 데이터를 가져온다
### 2. no_relation이 아니면서 + 외국어가 전혀 없으면서 + 각 엔티티가 문장에 한 번만 존재하는 경우에 역번역->엔티티가 유지될 경우에만 더해준다
### 3. 각 엔티티는 문장에서 한번만 나오므로 find로 index를 찾아 수정해준다.
### 4. (주) 특수문자 처리한다.
### 5. train/val split 하여 학습한다


## 특수문자도 바뀌는 경우가 있으니 이부분 더 고려해볼 것

In [67]:
import pandas as pd
import re
raw_df = pd.read_csv('/content/new_train_back_translation_input.csv')
raw_df
#/opt/ml/project2/level2_klue_nlp-level2-nlp-01/dataset/train/new_train_back_translation_input.csv

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당으로 재탄...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있...,"{'word': '아성다이소', 'start_idx': 10, 'end_idx': ...","{'word': '박정부', 'start_idx': 19, 'end_idx': 21...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32419,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32420,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32421,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia
32422,32469,화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버...,"{'word': '화순군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '구충곤', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree


In [75]:
# df=raw_df.loc[0:6000] #혜빈
# df=raw_df.loc[6001:12000] #인진
# df=raw_df.loc[12001:18000] #용우 
# df=raw_df.loc[18001:24000] #준원
# df=raw_df.loc[24001:32424] #현정
# df

,id,sentence,subject_entity,object_entity,label,source
24001,24024,또한 시리즈 구성은 《무한의 리바이어스》나 《허니와 클로버》 등으로 알려진 쿠로다 ...,"{'word': '코우가 윤', 'start_idx': 66, 'end_idx': ...","{'word': '만화가', 'start_idx': 62, 'end_idx': 64...",per:title,wikipedia
24002,24025,유진기업이 최첨단 기술이 적용된 스마트팩토리를 추진하는 배경은 유경선 유진그룹 회장...,"{'word': '유진그룹', 'start_idx': 39, 'end_idx': 4...","{'word': '유경선', 'start_idx': 35, 'end_idx': 37...",org:top_members/employees,wikitree
24003,24026,지난 19일 방송된 KBS '불후의 명곡2-전설을 노래하다' 배철수 편에서는 가수 ...,"{'word': '유회승', 'start_idx': 61, 'end_idx': 63...","{'word': '엔플라잉', 'start_idx': 56, 'end_idx': 5...",per:employee_of,wikitree
24004,24027,"유럽 위원회 의장 조제 마누엘 바호주, 아르메니아 대통령 세르지 사르키샨, 에스토니...","{'word': '세르지 사르키샨', 'start_idx': 32, 'end_idx...","{'word': '아르메니아', 'start_idx': 22, 'end_idx': ...",per:origin,wikipedia
24005,24028,"대한민국에서는 1990년에 금성사(현, LG전자)에서 비디오판으로 닌자 거북이라는 ...","{'word': '금성사', 'start_idx': 15, 'end_idx': 17...","{'word': 'LG전자', 'start_idx': 22, 'end_idx': 2...",org:alternate_names,wikipedia
...,...,...,...,...,...,...
32419,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32420,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32421,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia
32422,32469,화순군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버...,"{'word': '화순군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '구충곤', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree


In [ ]:
trans_list=[]
for i, sen, sub, obj, label, source in notebook.tqdm(zip(df['id'], df['sentence'], df['subject_entity'], df['object_entity'], df['label'], df['source'])):
    #no_relation이 아니면서 / 외국어가 전혀 없으면서 / 각 엔티티가 문장에 한 번만 존재하는 경우에
    if( label!="no_relation"):
        if(find_foreign('4e00', '9fff', sen)==False) and (find_foreign('0400', '04ff', sen)==False) and (find_foreign('0600', '06ff', sen)==False) and (find_foreign('0370', '03ff', sen)==False) and (find_foreign('3040', '309f', sen)==False) and (find_foreign('30a0', '30ff', sen)==False):
            sub = eval(sub)
            obj = eval(obj)
    
            sub_word=sub['word']
            obj_word=obj['word']
    
            sub_word_cnt=sen.count(sub_word)
            obj_word_cnt=sen.count(obj_word)
            
            if(sub_word_cnt==1 and obj_word_cnt==1): #역번역하자! #17658개에 대해서
                new_sen=back_trans(sen,"en")
                # new_sen=back_translation(sen,"en")
                if sub_word in new_sen and obj_word in new_sen: #번역시에도 문장의 엔티티가 잘 살아있는 경우에만 추가
                    ss=new_sen.find(sub_word)
                    se=ss+len(sub_word)-1
                    os=new_sen.find(obj_word)
                    oe=os+len(obj_word)-1
                    
                    sub['start_idx']=ss
                    sub['end_idx']=se
                    obj['start_idx']=os
                    obj['end_idx']=oe
                    
                    sub=str(sub)
                    obj=str(obj)
                    
                    temp=[]
                    temp.append(i)
                    temp.append(new_sen)
                    temp.append(sub)
                    temp.append(obj)
                    temp.append(label)
                    temp.append(source)
                    trans_list.append(temp)

0it [00:00, ?it/s]

또한 시리즈 구성은 《무한의 리바이어스》나 《허니와 클로버》 등으로 알려진 쿠로다 요스케이며, 캐릭터 디자인은 만화가 코우가 윤, 메카닉 디자인은 에비카와 카네타케, 야나세 타카유키, 테라오카 켄지, 후쿠치 히토시, 나카타니 세이치, 오오카와라 쿠니오 등이 담당했다.
유진기업이 최첨단 기술이 적용된 스마트팩토리를 추진하는 배경은 유경선 유진그룹 회장의 미래 레미콘 사업에 대한 방향과 전략에서 찾을 수 있다.
지난 19일 방송된 KBS '불후의 명곡2-전설을 노래하다' 배철수 편에서는 가수 바비킴, 김조한, 엔플라잉 유회승, 잔나비, 로맨틱펀치 등이 출연해 경연을 벌였다.
유럽 위원회 의장 조제 마누엘 바호주, 아르메니아 대통령 세르지 사르키샨, 에스토니아 대통령 토마스 헨드릭 일베스, 핀란드 대통령 사울리 니니스퇴, 발트 국가 교황 대사 및 교황 프란치스코, 리투아니아 대통령, 총리, 국회 의장 러시아 대통령 블라디미르 푸틴, 스웨덴 외무부 장관 칼 빌트, 터키 대통령 압둘라 귈, 독일 대통령 요아힘 가우크가 조의를 밝혔다.
대한민국에서는 1990년에 금성사(현, LG전자)에서 비디오판으로 닌자 거북이라는 제목으로 처음 발매 하였다.
2009년 9월 30일, 호나탄은 멕시코 국가대표팀 감독인 하비에르 아기레의 부름을 받아 콜롬비아와의 친선 경기에 선발 출장했다.
2013 시즌 끝나고 중국 슈퍼리그의 허난 젠예로 이적할 예정이었으나, 메디컬테스트에 문제가 생겨 다시 성남 FC로 복귀하였다.
현재는 찰스 2세의 후궁에게서 낳은 서자들과 찰스 1세의 후궁들에게서 낳은 서자들 중에서 백작작위를 받은 가문의 남계 후손이 일부 전하고 있다.
고흥군(군수 송귀근)은 지난 10일 이랜드재단과 취약계층 위기가정 지원을 위한 업무협약을 체결하여 최대 1억원을 지원할 계획이라고 밝혔다.
결국, 2016년 8월 31일에 토리노로 임대 이적을 확정지었는데, 조 하트는 1929년 세리에 A가 개막한 이후로 이탈리아에서 뛰는 첫 번째 잉글랜드 골키퍼가 되었다.
그레이가 작곡한 '119'

호세 안토니오 레예스 칼데론(1983년 9월 1일, 안달루시아 지방 우트레라 ~ 2019년 6월 1일)은 스페인의 프로 축구 선수였으며, 주로 좌측 미드필더로 활약하나, 공격수이기도 하였다.
2019년 자유당이 국민민주당에 합당되자 탈당하여 레이와 신센구미를 창설하고 총수가 되었다.
특히 아시아문화중심도시추진단장 경력을 통해 문화체육관광부와 정책적 호흡을 맞춰 문체부와 시, 국립아시아문화전당과 시, 시와 자치구간 정책협의회의 정례화를 이끌어내는데 큰 역할을 했으며 이를 바탕으로 문화전당의 주변 소상공인 지원과 인쇄의 거리 주변상가 상인들의 애로사항을 청취하고 해결책을 모색하는 등 등 현장중심 행정에 주력했다.
남양유업(회장 홍원식)이 신제품 ‘굿넛’의 광고기획자를 찾는다는 콘셉트로 수상작이 실제 광고 콘텐츠가 되는 ‘굿넛햇 UCC 공모햇’ 공모전을 진행한다고 8일 밝혔다.
크립톤 퓨처 미디어에서는 카가미네 린·렌의 10주년 공식 일러스트에 같은 회사의 제품인 하츠네 미쿠를 센터로 넣고, 카가미네 렌이 욱일기 배경에 알몸으로 앉아있으며 카가미네 린의 리본으로 다소 가려져 있는 모습으로 일러스트를 공개했다 .
2017년 2월 10일 장제원의 아들 NO:EL(장용준)이 Mnet에서 방영된 고등래퍼에 출연하면서 성매매 논란에 휩싸였다.
완투시합 2회(6월 18일 워싱턴 내셔널스의 라이언 짐머맨에게 리드오프 홈런을 허용, 2-3으로 패배했음)를 기록했다.
이후 형 여운형을 따라 조선인민당, 근로인민당에 참여했으나 1946년 박헌영의 노선에 반발하여 조선인민당을 탈당하여 사회민주당 당수 등을 지냈고, 1947년 12월 민족자주연맹 중앙집행위원으로 활동했다.
2015년 2분기에 방영한 후지 TV 월요일 밤 9시 드라마 《어서 오세요, 우리 집에》를 보고 주연이자 아라시의 멤버인 아이바 마사키가 예능도 드라마도 할 수 있는 모습이 멋지다고 여겨 동경하던 중, 어머니 친구의 권유로 2016년 6월 19일, 만 13세로 사무소에 입소하였다.
2002년 UAE 1부 리그의 알아인

대표적 산하단체로 학교로는 대한민국 충청남도 천안시 및 아산시에 있는 종합대학교인 선문대학교를 비롯해, 경기도 가평군의 선학유피대학원대학교와 청심국제중학교·청심국제고등학교, 서울특별시 광진구의 선화예술고등학교, 선화예술중학교와 경복초등학교, 서울특별시 은평구의 선정국제관광고등학교, 선정고등학교, 선정중학교 등이 있으며, 미국에는 통일신학대학원, 브리지포트대학교 등이 있다.
메디치 가문 출신에 또다른 교황인 클레멘스 7세와 껄끄러운 관계를 유지하였는데 코냑동맹전쟁중에 동맹군의 총사령관이면서도 전쟁에 소극적으로 임했고 1527년에 동맹국인 교황령 로마가 침공을 당했는데도 수수방관하였다.
'성폭행 혐의'로 구속기소된 배우 강지환씨(42·본명 조태규)에 대해 검찰이 징역 3년을 구형했다.
홍무제의 손자이며, 영락제와 인효문황후 서씨의 장남이다.
루벤 데 라 레드 구티에레스(1985년 6월 5일, 마드리드 지방 모스톨레스 ~)는 스페인의 전 축구 선수로, 이전에 중앙 미드필더로 활약했으며, 현재 감독이다.
문복희는 20일 유튜브 커뮤니티에 올린 글에서 "요즘 여러 먹방 유튜버들에 대한 의혹이 많고 저한테도 화살이 돌아와서 제가 해명글을 적었었다"며 입을 열었다.
마츠다 유사쿠(1949년 9월 21일 ~ 1989년 11월 6일)는 일본의 가수이자 배우이다.
권력 분점 협상에서 양측은 '무가베는 형식적인 국가원수로 남고, 모건 창기라이는 총리를 맡는 방안'으로 합의한 것으로 보였다.
그는 또한 경제학자이자 벨기에 기독교사회당의 당원이기도 하였다.
2016년 8월 24일 SK 와이번스와의 경기에서 김광현을 상대로 KBO 리그 통산 1390타점을 기록했고, KBO 리그 통산 최다 타점 신기록을 세웠다.
압둘 마지드 와리스는 가나의 프로 축구 선수로, 현재 프랑스 리그 1의 스트라스부르와 가나 국가대표팀에서 스트라이커로 활약하고 있다.
데뷔전 아이비는 JYP 엔터테인먼트의 연습생으로 있던중 대한민국의 가수 이수영의 〈꽃들은 지고〉의 뮤직비디오에 출연했다.
장흥고등학교(교장 김광

인천도시공사는 에너지절감 및 친환경 경영을 실천하고자 관용차량 일부를 한국GM에서 생산하는 전기자동차로 임차하고, 본사 주차장에 급속 전기차충전소를 구축해 24시간 개방 운영한다고 밝혔다.
시는 27일 이용섭 시장과 소영민 제31보병사단장, 박상규 한국전자통신연구원 부원장, 조문수 한국드론산업진흥협회장, 김지찬 LIG넥스원 대표이사가 참석한 가운데 시청 비즈니스룸에서 ‘국방드론 산업육성을 위한 업무협약’을 체결했다.
안영미 씨는 지난 2018년 송은이 씨, 김신영 씨, 신봉선 씨, 김영희 씨와 함께 ‘셀럽파이브’라는 개그우먼 걸그룹을 조성해 활동한 바 있다.
아단 막 가브란은 574년경-609년경의 달 리어타 국왕이다.
윤형관 하이에어 대표이사는 "전남 동부권 지역민의 교통 편익 증진과 함께 관광 산업 및 지역 경제 활성화에 도움을 드리는 항공사로 거듭나겠다"고 포부를 밝혔다.
김강윤(1995년 7월 11일 ~)은 대한민국의 리그 오브 레전드 프로게이머로 아이디는 Trick, 포지션은 정글라이너이며 현재 리그 오브 레전드 유러피언 챔피언십의 SK 게이밍에서 활동하고 있다.
그는 또 자유한국당이 박영선 중소벤처기업부 장관과 함께 공직선거법 위반 등의 혐의로 고발한 것에 대해 “제가 왜 고발을 당했는지 잘 이해가 안 된다"면서 "고발의 ‘무기화’”라고 비난했다.
배우 기태영(김용우)이 딸 로희에게 말로는 당해내지 못하는 일화를 털어놔 웃음을 자아냈다.
1917년에 블라디미르 레닌과 레프 트로츠키가 주도한 러시아 혁명 이후 소련에서는 동성애를 비범죄화 하였다.
또한 그는 우마이야 왕조의 왕자 칼리드 빈 야지드 빈 무아위야 와 시아파 6대 이맘인 자파르 알 사디크로부터도 교육을 받은 것으로 전해진다.
'2'의 주인공으로, '765 프로덕션'의 타카기 준지로 사장으로부터 소속 아이돌의 프로듀스를 의뢰받는다.
쿠빌라이는 칭기즈 칸의 손자이자 툴루이 칸과 소르칵타니 베키의 넷째 아들이며 몽케 칸의 동생이었다.
이 판례는 국제농구연맹(FIBA)의 마감 시한과 연관된 비슷한 

...

2014년 7월 17일, 울산 현대는 부산 아이파크의 양동현을 영입하는 조건으로 박용지와 김용태를 부산으로 이적시킨다고 발표하였다.
또한, 방위청이 방위성으로 승격하여 해외 파견이 주요 임무로 바뀌었다.
하지만 신덕왕후 강씨 소생 이방석을 세자로 책봉하는데 적극적으로 참여했고 "태조가 계룡산 터를 보고 돌아때에 정안대군 [[조선 태종|이방원]이 남은의 장막에 들어가니, 남은이 좋아하지 않으며 말하기를, ‘이제부터 내 장막에 들어오지 마시오.’ 하기에, 이방원이 드디어 나와서 들어가지 않았었다. 이때에 태조가 세자를 남은에게 부탁하시었다.”이로서 [[조선 태종|정안대군] 과 적을 지게 된다.
밀라그로스 카브랄 데 라 크루스(1978년 10월 17일 도미니카 공화국 산토도밍고 ~)는 도미니카 공화국의 전 여자 배구 선수이며, 스파이크 높이는 308cm, 블로킹 높이는 305cm이다.
바르톨로메오 그라데니고(1260년 – 1342년 12월 28일)는 제53대 베네치아 공화국의 도제이며, 1339년 11월 7일부터 사망할때까지 수행했다.
쿠바 혁명은 젊은 변호사 피델 카스트로, 에르네스토 게바라, 라울 카스트로 등의 사회주의 혁명가들이 두 차례에 걸친 무장 투쟁을 벌여 1959년 1월 1일 완수한 혁명을 말한다.
공격수 로빈 콰이손은 전반 27분 헤딩 패스를 받아 날카로운 오버헤드 킥으로 프라이부르크의 골문을 노렸으나 공은 골대를 살짝 빗나갔다.
청나라 조정에서는 좌종당, 장지동 등의 불만의 목소리가 들끓었고, 격노한 서태후는 1880년에 귀국한 숭후를 사형시키려 했다.
해남군(군수 명현관)은 신종 코로나바이러스 감염증 발생으로 어려움을 겪는 납세자가 있을 경우 납부기한 연장, 징수유예 등 세제지원을 실시한다.
후에 에드워드 2세의 아들인 에드워드 3세가 1343년에 자신의 장남이며 제1왕위계승자인 에드워드에게 웨일스 공의 칭호를 내리며, 장남이 잉글랜드의 왕위를 잇기 전에 사망하면서 칭호를 다음 왕위계승자가 될 장남의 차남 리처드에게 넘긴다.
바른미래당 13명의 비

11월 1일 국방위 국정감사에서는 국방부의 다른 사령부인 기무사·정보사 요원의 정치글 의혹이 제기됐다.
이어진 승부차기에서 양팀 1번키커인 박은선과 이세은이 성공시켰으며, 이천 대교의 2번 키커인 쁘레치냐의 슛이 골포스트를 맞고 나왔고 이어 인천 현대제철의 김도연이 성공시킴으로써 승부차기에서 1:2로 인천 현대제철이 앞서기 시작했다.
2009년에는 일본으로 건너가 일본 프로 야구 센트럴 리그 팀 요미우리 자이언츠에서 지도자 연수를 받으며 2군 타격 코치로 1년간 몸담은 후 귀국했다.
김지수는 충청북도 청주시에서 태어나 세한대학교 실용음악과를 다니던 도중 2010년 서바이벌 오디션 프로그램인 Mnet 《슈퍼스타K 시즌2》에 제주지역 예선에 참가하여 I'm yours와 초콜릿 드라이브를 불러 심사위원의 호평을 받았다.
오토 2세와 테오파누 공주의 혼인식은 972년 4월 14일 로마에서 요한 13세의 집전 아래 거행되었다.
고등학교를 졸업한 후, 미국에 유학하였다가 오스트레일리아 시드니 대학교에서 공학 석사를 취득하였다.
김강립 중앙재난안전대책본부 총괄대변인(보건복지부 1차관)은 16일 열린 정례 브리핑에서 “검사량이나 결과를 인위적으로 조정하려면 모든 선별진료소와 검사기관의 인력이 조작에 협조해야 가능한 일”이라며 “이는 현실적으로 결코 있을 수 없는 일”이라고 강조했다.
프랑스 앙리 2세와의 전쟁에서 거듭 패배한 황제 카를 5세는 동생 페르디난트 1세가 체결한 파사우 조약을 인정하고, 동생에게 독일의 통치권을 넘기고 물러났다.
김종갑 한국전력공사 사장도 전력망 구축에 대해 긍정적으로 검토하겠다는 뜻을 내비쳤다.
이날 회동에는 국내 5대 그룹 중 삼성 이재용, 현대차 정의선, SK 최태원, 롯데 신동빈 회장이 참석했으며, LG그룹은 권영수 회장이 구광모 회장을 대신했다.
정태옥의 그 망언 때문에 당시 인천광역시장 후보로 출마한 유정복 후보는 더불어민주당 박남춘 후보에게 무려 20% 넘는 격차로 대패하며 낙선의 고배를 마신 바 있었다.
빌헬름 푸르트벵글러는 독일의 

조선대학교 SW융합교육원은 에너지 ICT 공기업인 한전KDN(사장 박성철)과 전남도교육청, 조선대학교 공동 주관으로 SW교육 저변확대 프로젝트인 '2019년 SW코딩교육 주말교육기부활동'을 성황리에 마쳤다고 말했다.
한국씨티은행은 대한민국 서울특별시 중구 다동에 본사를 두고 있는 은행이다.
광주광역시(시장 이용섭)는 코로나19 고위험군으로 보건소에서 의료비를 지원받고 있는 성인암 환자와 희귀질환자에게 19일부터 마스크를 배부한다.
988년에는 키예프 루스의 블라디미르 1세 대공이 케르소네소스에서 기독교 세례를 받았다.
김선빈은 KBO 리그 KIA 타이거즈의 내야수이다.
SPC그룹(회장 허영인)이 운영하는 파인 캐주얼 브랜드 ‘쉐이크쉑’이 국내 론칭 3주년을 맞이해 ‘제로컴플렉스 버거’를 한정판으로 출시하고 다양한 이벤트를 진행한다고 16일 밝혔다.
7월 7일 예정되어 있던 J리그 중 J2리그 22라운드 경기 중 교토 니시교고쿠 육상경기장에 열리는 교토 상가 FC 대 아비스파 후쿠오카 1경기, J3리그 16라운드 경기 중 돗토리 은행 버드 스타디움에 열리는 가이나레 돗토리 대 더스파구사쓰 군마 경기, 미쿠니 월드 스타디움 기타큐슈에 열리는 기라반츠 기타큐슈 대 가고시마 유나이티드 FC 2경기가 호우의 영향으로 중지, 연기되었다.
엘사 파타키와 크리스 헴스워스 사이에는 2012년 5월에 태어난 딸인 인디아 로즈, 2014년 3월에 태어난 쌍둥이 아들인 트리스탄, 사샤가 있다.
레반테 UD는 발렌시아에 위치한 발렌시아 시립 경기장 경기장을 근거로 하는 스페인의 축구 클럽이다.
2004년, 호비뉴는 37경기 출전 21골로 1년을 마쳤고, 산투스는 또다시 캄페오나투 브라질레이루 정상에 올랐다.
하인리히 3세는 독일 잘리어 왕가의 두번째 국왕이자 신성로마제국 황제이다.
교황 요한 23세는 1962년 요셉의 이름을 로마 가톨릭 미사 전문에 실리도록 했으며, 제2차 바티칸 공의회의 성공적인 개최를 위하여 특별히 요셉의 도움을 위탁하였다.
이번에 영입된 선수는 피닉스 선스에서

광주대 강유림은 9일 인천 하나 글로벌캠퍼스에서 열린 2019-2020 한국여자농구연맹(WKBL) 신입 선수 선발회에서 2라운드 3순위로 하나은행에 지명됐다.
잣쇼노쿠마 역은 일본 후쿠오카현 후쿠오카시 하카타구에 위치한 서일본 철도 덴진오무타 선의 역이다.
이 대회에서 주목할 만한 선수는 핀란드의 파보 누르미로 단체에서 3개의 금메달을 개인에서 1500 m와 5000 m에서 2개의 금메달을 목에 걸었다.
에드워드 5세는 잉글랜드의 왕 에드워드 4세와 엘리자베스 우드빌 사이의 장남으로, 아버지가 잠시 추방당하였을 때인 1470년 11월 2일 웨스트민스터 사원의 성역에서 태어났다.
조 후보자는 그러나 동생의 이혼과 관련 김진태 자유한국당 의원이 어제 조 후보자의 부친 묘비 사진까지 공개한 것과 관련 "선친의 묘소까지 찾아가서 비석 사진을 찍어 손자, 손녀 등의 이름을 공개하는 것은 개탄스럽다"고 발끈했다.
신도 하루이치는 일본의 남자 음악가로 포르노그라피티의 기타리스트이다.
여당인 정의개발당 (AKP)은 4차 내각 연임을 추진했다.
또한 윤장배의 1년 후배는 이수만 현 SM 엔터테인먼트 이사이고, 가수 비를 발굴한 박진영 JYP 엔터테인먼트 대표의 외삼촌이기도 하다.
그 때 당시 팀은 기존 3루수였던 안재만을 FA를 선언해 이적한 김재현의 보상 선수로 LG 트윈스에 보냈기 때문에 그가 3루수를 차지할 수 있었다.
만약 고구려 왕이나 을지문덕이 오면 붙잡으라는 양제의 밀지를 받았었기에 잡으려고 했는데, 위무사로 종군하고 있던 상서 우승 유사룡이 굳이 말려서 그만두고 돌아가게 하였다.
고흥군(군수 송귀근)은 부동산거래신고법이 일부 개정되면서 올해 2월 21일부터 부동산 실거래신고 기한이 60일에서 30일로 단축됨에 따라 달라지는 내용에 대한 홍보 활동에 나섰다.
그 후 1933년 한스 베테와 아르놀트 조머펠트에 의해 양자역학이 반영된 결과로 드루드 모형은 확장되었다.
개인적 상연의 기간들에서 제임스는 우수한 해를 가졌으며, NBA 정규 시즌에서 최고 평균인 한 경기에

이에 라인스 프리버스 공화당 대표는 알래스카 주당의 당규가 다른 지역의 당규와 달리 경선에서 모든 후보가 사퇴하고 한 후보만 남은 경우 사퇴한 후보들의 표는 모두 남은 한 후보에게 몰아주도록 규정하고 있어 이에 따른 것이라고 해명하고 개표 기록의 수정을 거부하였다.
백제의 멸망에 대한 내용이 가사에 담겨 있는 이 노래는 조선총독부가 발매금지 조치를 취했고, 후에 작사가 조명암이 조선민주주의인민공화국으로 가면서 대한민국에서도 금지곡이 되었다.
박삼득 국가보훈처장은 보훈가족과 국민이 체감하는 빈틈없는 보훈정책 구현을 위해 26일 인천보훈지청을 방문하여 직원들과 소통의 시간을 가졌다.
2006년부터 2010년까지는 영국 최다부수신문인 데일리 텔레그래프가 소속된 텔레그래프 미디어 그룹의 총편집장으로 활동하였다.
김국찬(1996년 2월 15일~)은 대한민국의 농구선수이며 포지션은 스몰 포워드이다.
그러나, 그의 사나운 초연 동안, 그리고 마가렛 대처와 존 메이저의지도하에 보수당은 혁명을 강력하게 반대하고 1997 년 혁명 국민 투표에서 그 반대에 반대했다.
경주 이씨 이승훈은 고려 시대 평장사를 지냈고, 차자는 이주좌는 아산 이씨로 분적하였다.
1855년 카를 프리드리히 가우스의 후임으로 괴팅겐 대학교에서 고등수학 교수를 맡았다.
이날 의식에는 토마스 바흐 국제올림픽위원회 위원장, 그리스 올림픽 위원회 위원장, 올림피아 시장, 한국 측 대표로는 이낙연 국무총리를 비롯한 주요 인사들이 참석하였다.
그의 딸 칼 폴라니-레빗은 캐나다 몬트리올의 맥길 대학교의 명예 교수이다.
성남 일화 천마의 연고지 사태와 관련된 KBS 추적 60분 2001년 3월 25일 방송분에서 성남시 축구협회장에게 1999년 9월 당시 부천 SK 단장으로부터 전화가 와서 안양 LG가 앞으로 상암구장 사용을 신청하니 우리 부천 SK도 서울 상암구장 사용 신청을 안 할 수가 없다는 발언을 하였으며2000년 6월 당시 안양 LG 치타스 단장은 연맹에서 기존 구단의 서울 입성을 원칙적으로 반대하지 않으니 서울

In [30]:
new_df = pd.DataFrame(trans_list)
new_df
new_df.to_csv('/content/new_train_back_translation_input_copy_new.csv',index=False)

## entity가 문장에 없으면 출력

In [ ]:
for sen, sub, obj in zip(df['sentence'], df['subject_entity'], df['object_entity']):
    sub = eval(sub)
    obj = eval(obj)
    sub_word=sub['word']
    obj_word=obj['word']
    if sub_word not in sen:
        print(sen)
        print(sub_word)
    if obj_word not in sen:
        print(sen)
        print(obj_word)

## index에 문제 있으면 출력

In [ ]:
for sen, sub, obj in zip(df['sentence'], df['subject_entity'], df['object_entity']):
    raw_sub = eval(sub)
    raw_obj = eval(obj)
    ss=raw_sub['start_idx']
    se=raw_sub['end_idx']
    os=raw_obj['start_idx']
    oe=raw_obj['end_idx']
    if(sen[ss:se+1]!=raw_sub['word']):
        print(sen[ss:se+1])
        print(raw_sub['word'])
        
    if(sen[os:oe+1]!=raw_obj['word']):
        print(sen[os:oe+1])
        print(raw_obj['word'])